<a href="https://colab.research.google.com/github/onshmidi21/ia/blob/main/vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
# Utiliser le GPU si disponible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [3]:
# Redimensionnement à 224x224 (taille attendue par VGG16) et normalisation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [4]:
# Datasets CIFAR-100
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)


100%|██████████| 169M/169M [00:03<00:00, 46.3MB/s]


In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


In [6]:
vgg16_pretrained = torchvision.models.vgg16(pretrained=True)
vgg16_pretrained.classifier[6] = nn.Linear(4096, 100)  # Adapter à CIFAR-100
vgg16_pretrained = vgg16_pretrained.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 81.7MB/s]


In [7]:
optimizer = torch.optim.SGD(vgg16_pretrained.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()


In [8]:
number_of_epochs = 5

for epoch in range(number_of_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = vgg16_pretrained(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}: Loss: {running_loss / len(train_loader):.4f}')


Epoch 1: Loss: 2.1822
Epoch 2: Loss: 1.3598
Epoch 3: Loss: 1.0325
Epoch 4: Loss: 0.8033
Epoch 5: Loss: 0.6144


In [9]:
torch.save(vgg16_pretrained.state_dict(), 'vgg16_cifar100.pth')


In [11]:
# Mettre le modèle en mode évaluation
vgg16_pretrained.eval()

correct, total = 0, 0

# Désactiver le calcul des gradients pour économiser de la mémoire
with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = vgg16_pretrained(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Affichage de la précision
accuracy = 100 * correct / total
print(f'Accuracy on test images: {accuracy:.2f}%')


Accuracy on test images: 67.67%
